In [1]:
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from fbprophet import Prophet
import plotly
from fbprophet.plot import plot_plotly, plot_components_plotly
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy import signal
import numpy as np
import altair as alt

In [2]:
df = pd.read_csv('MentalHealth_data/raw_data_2017/01Jan2017.csv').drop(['Unnamed: 0'], axis = 1)
for year in range(2017,2021):
    for file in os.listdir(f'MentalHealth_data/raw_data_{year}'):
        if file[-3:] == 'csv':
            df_new = pd.read_csv(f'MentalHealth_data/raw_data_{year}/{file}').drop(['Unnamed: 0'], axis=1)
            df_new.time = df_new.time.apply(lambda x: file[:-4])
            df = pd.concat([df, df_new]).reset_index(drop=True)
df.head()


,text,time,score
0,"I don't know how many of you read it, but a fe...",01Jan2017,6
1,The reason that it occurred to me that I may h...,01Jan2017,1
2,Sometimes I don't really feel like a participa...,01Jan2017,5
3,"Hi, and thanks for reading this post. I'm look...",01Jan2017,1
4,"Loaded question I know, trying to scribble thi...",01Jan2017,7


In [3]:
df.time = df.time.apply(lambda x: datetime.strptime(x, '%d%b%Y'))
df = df.sort_values('time')

In [4]:
dates = sorted([datetime.strftime(datetime.strptime(file[:-4], '%d%b%Y'), '%Y-%m-%d') \
                for year in range(2017, 2021) \
                for file in os.listdir(f'MentalHealth_data/raw_data_{year}') \
                if file[-3:] == 'csv'])
data = []
df_text = df[df['text'] != '[deleted]']

for date in dates:
    data.append(df_text[df_text['time'] == datetime.strptime(date, '%Y-%m-%d')])

In [5]:
#prophet expects a dataframe with (date, value)
prophet_input = pd.DataFrame(columns = ['ds','y'])
analyzer = SentimentIntensityAnalyzer()
for i in tqdm(range(0, len(data))):
    texts = data[i]['text'].values
#     if (len(texts) == 0): 
#         continue
    date = data[i]['time'].iloc[0] #date will be the same for all values in list, just pick first
    #iterate over the posts in the texts list
    for j in texts:
        try:
            vs = analyzer.polarity_scores(j)
            new_data = pd.DataFrame([[date, vs['compound']]], columns = ['ds','y'])
            prophet_input = pd.concat([prophet_input, new_data], ignore_index = True)
        except:
            print("We couldn't process this post because it was: ", j)
            continue


100%|██████████| 732/732 [20:58<00:00,  1.72s/it]


In [10]:
#need to split dataframe that was created above into four chunks (based on year) and then groupby date
data_2017 = prophet_input[prophet_input['ds'].dt.year == 2017] 
data_2018 = prophet_input[prophet_input['ds'].dt.year == 2018] 
data_2019 = prophet_input[prophet_input['ds'].dt.year == 2019] 
data_2020 = prophet_input[prophet_input['ds'].dt.year == 2020] 
grouped_2017 = data_2017.groupby([data_2017['ds'].dt.date]).mean()
grouped_2018 = data_2018.groupby([data_2018['ds'].dt.date]).mean()
grouped_2019 = data_2019.groupby([data_2019['ds'].dt.date]).mean()
grouped_2020 = data_2020.groupby([data_2020['ds'].dt.date]).mean()
df_2017 = grouped_2017.reset_index()
df_2018 = grouped_2018.reset_index()
df_2019 = grouped_2019.reset_index()
df_2020 = grouped_2020.reset_index()
df_all_years = pd.concat([df_2017,df_2018,df_2019,df_2020])
df_all_years.set_index('ds', inplace = True)

In [11]:
freq = df_all_years.reset_index().rename(columns ={'ds': 'date', 'y':'y'})
#get rolling average of all data and plot it overtime
smoothed_vader_score = df_all_years.rolling(7).mean().reset_index().rename(columns ={'ds': 'date', 'y':'y'})
#convert to datetime because altair was being weird
freq["date"] = pd.to_datetime(freq["date"])
smoothed_vader_score["date"] = pd.to_datetime(smoothed_vader_score["date"])

series = alt.Chart(freq).mark_line(opacity=0.3,
                                  ).encode(x='date', y='y'
                                  ).properties(title='VADER compound score of post on r/MentalHealth'
                                  ).interactive(bind_y=False)

smoothed = alt.Chart(smoothed_vader_score.reset_index()).mark_line(
                                                           ).encode(x='date', y='y'
                                                           ).properties(title='Vader score of r/MentalHealth Posts'
                                                           ).interactive(bind_y=False)

chart = (series + smoothed).properties(width=800, height=300)
chart.save('figures/VADER_MentalHealth.html')


In [72]:
df_all = pd.concat([data_2017,data_2018,data_2019,data_2020])
df_low_VADER = df_all[df_all['y'] < -0.1]
posts_per_day_low_VADER = df_low_VADER.groupby([df_low_VADER['ds'].dt.date]).count()
posts_per_day = df_all.groupby([df_all['ds'].dt.date]).count()
normalized_VADER_count = (posts_per_day_low_VADER/posts_per_day).y
a = normalized_VADER_count.index.to_series()


df = pd.DataFrame({'date': a, 'count': normalized_VADER_count}).reset_index().drop(columns=['ds'])
df['date'] = pd.to_datetime(df['date'])

VADER = alt.Chart(df).interactive(bind_y=False).mark_line(opacity=0.3).encode(x='date',y='count')
chart = VADER.properties(width=800, height=300)
chart.save('figures/Frequency_of_low_VADER.html')




#try to import number of subscribers 
trend = pd.read_csv('subscriber_count.csv')
trend['subscribers'] = trend['subscribers'].interpolate()
trend['date'] = trend['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M').replace(hour=0, minute=0))

overlap = pd.DataFrame()
dates = freq[freq['date'].isin(trend.date)]['date']
overlap['date'] = dates
overlap_subs = trend[trend['date'].isin(dates)].reset_index().drop(['index'], axis=1)
overlap_subs_series = 
posts_per_day_low_VADER_series = posts_per_day_low_VADER.y
print(posts_per_day_low_VADER_new)
normalized_VADER_subscribers = (posts_per_day_low_VADER.y/overlap_subs)


          date  subscribers
0   2017-01-01      22354.0
1   2017-01-03      22439.0
2   2017-01-05      22514.0
3   2017-01-07      22609.0
4   2017-01-09      22685.0
..         ...          ...
727 2020-12-22     210962.0
728 2020-12-24     211286.0
729 2020-12-26     211713.0
730 2020-12-28     212149.0
731 2020-12-30     212625.0

[732 rows x 2 columns]
ds
2017-01-01     30
2017-01-03     12
2017-01-05     11
2017-01-07     11
2017-01-09     15
             ... 
2020-12-22    134
2020-12-24    148
2020-12-26    138
2020-12-28    184
2020-12-30    147
Name: y, Length: 732, dtype: int64


TypeError: cannot perform __rtruediv__ with this index type: DatetimeArray

## Everything below this line is for predicting with Prophet, TBD if this will still even be used

In [ ]:
#periodgram of data
f, Pxx_den = signal.periodogram(df_all_years['y'])
plt.semilogy(f, Pxx_den)
plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

In [ ]:
#start fitting data using Prophet model
m_2017 = Prophet(weekly_seasonality=False)
#fit model with 2017 data, use the to predict 2018 data
m_2017.fit(df_2017)
future_2018 = m_2017.make_future_dataframe(periods=365)[-365:]
forecast_2018 = m_2017.predict(future_2018)
y_hat_2018 = forecast_2018[['ds','yhat']]
fig1 = m_2017.plot(forecast_2018)
#m_2017.plot_components(forecast_2018)

In [ ]:
#plot predicted_2018 vs true_2018 to check for similarities
plt.plot(y_hat_2018['ds'], y_hat_2018['yhat'])
plt.plot(df_2018['ds'],df_2018['y'])
#find correlation between the two values as well
y_hat_2018_trim = y_hat_2018.iloc[::2]
print(np.corrcoef(y_hat_2018_trim['yhat'], df_2018['y']))

In [ ]:
m_2018 = Prophet(weekly_seasonality=False)
#fit model with 2018 data, use the to predict 2019 data
m_2018.fit(df_2018)
future_2019 = m_2018.make_future_dataframe(periods=365)[-365:]
forecast_2019 = m_2018.predict(future_2019)
y_hat_2019 = forecast_2019[['ds','yhat']]
y_hat_2019 = y_hat_2019.iloc[::2]
fig2 = m_2018.plot(forecast_2019)

In [ ]:
#plot predicted_2019 vs true_2019 to check for similarities
plt.plot(y_hat_2019['ds'], y_hat_2019['yhat'])
plt.plot(df_2019['ds'],df_2019['y'])
#find the correlation between y_hat and y_true
y_hat_2019_trim = y_hat_2019.iloc[::2]
np.corrcoef(y_hat_2019['yhat'], df_2019['y'])

In [ ]:
m_2019 = Prophet(weekly_seasonality=False)
#fit model with 2018 data, use the to predict 2019 data
m_2019.fit(df_2019)
future_2020 = m_2019.make_future_dataframe(periods=365)[-365:]
forecast_2020 = m_2019.predict(future_2020)
y_hat_2020 = forecast_2020[['ds','yhat']]
fig3 = m_2019.plot(forecast_2020)

In [ ]:
plt.plot(y_hat_2020['ds'], y_hat_2020['yhat'])
plt.plot(df_2020['ds'],df_2020['y'])
#find the correlation between y_hat and y_true
y_hat_2020_trim = y_hat_2020.iloc[::2]
np.corrcoef(y_hat_2020_trim['yhat'], df_2020['y'])

In [ ]:
#try fitting 2017-2019 to predit 2020
df_pre2020 = pd.concat([df_2017,df_2018,df_2019])
m_2020 = Prophet(weekly_seasonality=False, yearly_seasonality=True)
m_2020.fit(df_pre2020)
future_all = m_2020.make_future_dataframe(periods=365)[-365:]
forecast_2020_all = m_2020.predict(future_all)
y_hat_2020_all = forecast_2020_all[['ds','yhat']]
fig4 = m_2020.plot(forecast_2020_all)

In [ ]:
plt.plot(y_hat_2020_all['ds'], y_hat_2020_all['yhat'])
plt.plot(df_2020['ds'],df_2020['y'])
#find correlation between true and predicted value
y_hat_2020_all_trim = y_hat_2020_all.iloc[::2]
np.corrcoef(y_hat_2020_all_trim['yhat'], df_2020['y'])

In [ ]:
#TODO: Check for correlation between these two years
#TODO: investigate changepoints of the prophet model
#TODO: Try to smooth initial data more
